In [1]:
import pandas as pd
import numpy as np
import math

df = pd.read_csv("churn_analysis.csv", index_col=0)
df.head()

,state,account_length,area_code,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,eve_mins,...,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,custserv_calls,treatment,mes_estim,churn
0,KS,128,415,no,yes,25,265.1,110,45.07,197.4,...,244.7,91,11.01,10.0,3,2.70,1,1,0.65,False.
1,OH,107,415,no,yes,26,161.6,123,27.47,195.5,...,254.4,103,11.45,13.7,3,3.70,1,0,0.55,False.
2,NJ,137,415,no,no,0,243.4,114,41.38,121.2,...,162.6,104,7.32,12.2,5,3.29,0,0,0.72,False.
3,OH,84,408,yes,no,0,299.4,71,50.90,61.9,...,196.9,89,8.86,6.6,7,1.78,2,1,0.28,False.
4,OK,75,415,yes,no,0,166.7,113,28.34,148.3,...,186.9,121,8.41,10.1,3,2.73,3,2,0.45,False.


In [3]:
from scipy.stats import chi2_contingency

control_group = df[df["treatment"] == 1]

len(control_group)

1097

In [5]:
cg = control_group

#Выбираем штаты
states = pd.unique(cg["state"])
pairs = []

for i in range(len(states) - 1):
    for j in range(i+1, len(states)):
        pairs.append((states[i], states[j]))

len(pairs)

1275

In [6]:
cg["churn"] = cg["churn"].map(lambda x: True if x == "True." else False)
cg["churn"]

C:\bin\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


0       False
3       False
8       False
12      False
17      False
        ...  
3319    False
3322     True
3323     True
3325    False
3330    False
Name: churn, Length: 1097, dtype: bool

In [32]:
#Q1
q1df = pd.DataFrame({"pairs": pairs})

subtable = []
chi2 = []
i = 0
for pair in pairs:
    a = len(cg[(cg["state"] == pair[0]) & (cg["churn"] == False)])
    b = len(cg[(cg["state"] == pair[1]) & (cg["churn"] == False)])
    c = len(cg[(cg["state"] == pair[0]) & (cg["churn"] == True)])
    d = len(cg[(cg["state"] == pair[1]) & (cg["churn"] == True)])
    subtable.append([[a,b],[c,d]])
    try:
        chi2.append(chi2_contingency([[a,b],[c,d]], correction=False)[1])
    except:
        chi2.append(999)
        
alpha = 0.05

q1df["subtable"] = subtable
q1df["chi2"] = chi2
    
q1df

,pairs,subtable,chi2
0,"(KS, OH)","[[18, 22], [7, 2]]",0.075524
1,"(KS, LA)","[[18, 15], [7, 0]]",0.024052
2,"(KS, IA)","[[18, 13], [7, 2]]",0.282190
3,"(KS, VT)","[[18, 18], [7, 5]]",0.616769
4,"(KS, CO)","[[18, 17], [7, 5]]",0.679125
...,...,...,...
1270,"(IL, HI)","[[11, 12], [1, 1]]",0.952933
1271,"(IL, SC)","[[11, 16], [1, 5]]",0.267505
1272,"(ND, HI)","[[21, 12], [3, 1]]",0.652998
1273,"(ND, SC)","[[21, 16], [3, 5]]",0.322191


In [33]:
#Q1

len(q1df[q1df["chi2"] < alpha])   #SUCCESS!

34

In [36]:
#Q2 применение не обосновано, интерпретация некорректна SUCCESS

#Q3
subtable = []
chi2 = []
i = 0
for pair in pairs:
    a = len(cg[(cg["state"] == pair[0]) & (cg["churn"] == False)])
    b = len(cg[(cg["state"] == pair[1]) & (cg["churn"] == False)])
    c = len(cg[(cg["state"] == pair[0]) & (cg["churn"] == True)])
    d = len(cg[(cg["state"] == pair[1]) & (cg["churn"] == True)])
    subtable.append([[a,b],[c,d]])
    try:
        chi2.append(chi2_contingency([[a,b],[c,d]], correction=True)[1])
    except:
        chi2.append(999)
        
alpha = 0.05

q1df["subtable"] = subtable
q1df["chi2_true"] = chi2
    
len(q1df[q1df["chi2_true"] < alpha])
#количество раво нулю, в среднем увеличивает, в среднем больше FAIL
#количество равно нулю, в среднем больше SUCCESS

0

In [35]:
q1df["chi2"].mean(), q1df["chi2_true"].mean()

(0.501827379873916, 0.6640566382051037)

In [40]:
#Q4
from scipy.stats import fisher_exact

fisher = []
for subtable in q1df.subtable.tolist():
    fisher.append(fisher_exact(subtable)[1])

q1df["fisher"] = fisher
len(q1df[q1df["fisher"] < alpha]), q1df["fisher"].mean()

#в среднем больше, чем хи-квадрат без поправки, меньше, чем хи-квадрат с поправкой FAIL
#<еще какая-то галочка нужна, хз>

(10, 0.6483383060020688)

In [43]:
#Q5-Q6

q5df = df[["day_calls", "mes_estim"]]

q5df.corr(method="pearson"), q5df.corr(method="spearman")

#Q5 - отрицательно, значимо SUCCESS
#Q6 - положительно, не значимо SUCCESS
#Q7 - необходимо взглянуть на данные глазами SUCCESS

(           day_calls  mes_estim
 day_calls   1.000000  -0.051794
 mes_estim  -0.051794   1.000000,            day_calls  mes_estim
 day_calls    1.00000    0.04335
 mes_estim    0.04335    1.00000)

In [46]:
#Q8 Крамер
# Таблица сопряженности имеет число столбцов равное числу штатов и две строки (churn == False, churn == True)

K_false = []
K_true  = []
N = 0 #общее число
for state in cg.state.unique():
    f = len(cg[(cg["state"] == state) & (cg["churn"] == False)])
    t = len(cg[(cg["state"] == state) & (cg["churn"] == True)])
    K_false.append(f)
    K_true.append(t)
    N += f+t

V_kramer =  math.sqrt(chi2_contingency([K_false, K_true])[0]/N)
V_kramer

#Не можем положиться SUCCESS

#Q9 статистически незначимо различие между 0 и 2 FAIL

0.20039321502033322